In [1]:
%matplotlib inline

In [2]:
!ls data/retailhero-uplift.zip 2>&1 > /dev/null || \
 wget https://storage.yandexcloud.net/datasouls-competitions/x5-retailhero/retailhero-uplift.zip -O data/retailhero-uplift.zip

In [3]:
!ls data/products.csv 2>&1 > /dev/null || \
 unzip -o data/retailhero-uplift.zip

In [4]:
!ls data/

clients.csv                  uplift_sample_submission.csv
products.csv                 uplift_test.csv
purchases.csv                uplift_train.csv
retailhero-uplift.zip


In [5]:
import pandas as pd

In [6]:
df_clients = pd.read_csv('data/clients.csv')
df_clients.head(1)

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U


In [7]:
df_products = pd.read_csv('data/products.csv')
df_products.head(1)

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.4,0,0


In [8]:
df_purchases = pd.read_csv('data/purchases.csv')
df_purchases.head(1)

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN


In [9]:
df_train = pd.read_csv('data/uplift_train.csv')
df_train.head(1)

,client_id,treatment_flg,target
0,000012768d,0,1


In [10]:
df_test = pd.read_csv('data/uplift_test.csv')
df_test.head(1)

,client_id
0,000048b7a6


In [11]:
!head -n2 data/uplift_sample_submission.csv

client_id,uplift
008fb49e3a,0.1149912020897228


In [12]:
df_products.describe()

,segment_id,netto,is_own_trademark,is_alcohol
count,41466.000000,43035.000000,43038.000000,43038.000000
mean,153.918222,0.536966,0.035178,0.055602
std,82.271376,8.274367,0.184232,0.229154
min,1.000000,0.000000,0.000000,0.000000
25%,105.000000,0.150000,0.000000,0.000000
50%,148.000000,0.300000,0.000000,0.000000
75%,214.000000,0.500000,0.000000,0.000000
max,321.000000,1150.000000,1.000000,1.000000


In [14]:
df_purchases.describe()

,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red
count,4.578657e+07,4.578657e+07,4.578657e+07,4.578657e+07,4.578657e+07,4.578657e+07,4.578657e+07,3.043356e+06
mean,8.049876e+00,6.084099e-02,-5.312626e+00,-3.181049e-01,7.775215e+02,1.247245e+00,7.348835e+01,7.677407e+01
std,1.268503e+01,2.426226e+00,3.603646e+01,3.287976e+00,7.965350e+02,3.137584e+00,8.753977e+01,8.427114e+01
min,0.000000e+00,0.000000e+00,-5.066000e+03,-3.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.860000e+02,1.000000e+00,3.000000e+01,3.100000e+01
50%,3.800000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.390000e+02,1.000000e+00,5.100000e+01,5.500000e+01
75%,1.030000e+01,0.000000e+00,0.000000e+00,0.000000e+00,9.760000e+02,1.000000e+00,9.000000e+01,9.500000e+01
max,2.399000e+03,3.000000e+02,0.000000e+00,0.000000e+00,3.514904e+04,1.494100e+04,3.514900e+04,8.789000e+03


In [15]:
df_clients.describe()

,age
count,400162.000000
mean,46.488112
std,43.871218
min,-7491.000000
25%,34.000000
50%,45.000000
75%,59.000000
max,1901.000000


In [16]:
df_purchases['transaction_datetime'].min()

'2018-11-21 21:02:33'

In [17]:
df_purchases['transaction_datetime'].max()

'2019-03-18 23:40:03'

In [18]:
df_purchases.groupby('client_id').agg({'transaction_datetime': ['min', 'max']})

transaction_datetime                     
                            min                  max
client_id                                           
000012768d  2018-12-01 07:12:45  2019-03-14 15:01:47
000036f903  2018-11-28 10:48:36  2019-03-17 10:29:37
000048b7a6  2018-12-15 10:33:11  2019-03-12 10:28:19
000073194a  2018-11-25 10:56:20  2019-03-16 09:41:21
00007c7133  2018-11-24 15:09:24  2019-03-04 16:23:30
00007f9014  2018-11-23 14:50:12  2019-03-15 18:34:21
0000a90cf7  2018-11-23 15:40:35  2019-03-15 15:32:09
0000b59cec  2018-11-22 16:11:13  2019-03-18 16:35:23
0000bb4e4e  2018-12-12 14:54:44  2019-03-09 17:03:40
0000bcec9c  2018-11-28 12:12:51  2019-03-16 16:59:40
0000eecb82  2018-12-26 09:53:26  2019-03-16 13:40:27
0000f0ecdb  2018-11-27 06:04:46  2019-03-05 06:04:40
00010925a5  2018-11-25 10:56:18  2019-03-08 07:09:50
0001f552b0  2018-11-24 07:14:43  2019-03-16 10:54:02
00020e7b18  2018-11-23 08:21:26  2019-03-15 12:04:23
000220a0a7  2018-12-11 16:54:16  2019-03-12 16:44:43
00022fd34f  2018-11-23 14:30:05  2019-03-16 07:59:56
0002ce2217  2018-12-09 13:09:48  2019-03-17 09:31:56
00031cbbe6  2019-01-17 17:43:17  2019-03-14 11:23:34
0003363542  2018-12-04 05:42:56  2019-02-27 15:52:42
00034f1549  2018-12-03 15:08:14  2019-03-06 16:02:15
00035a21d9  2019-02-09 11:46:22  2019-03-02 12:18:35
00037a9650  2018-12-06 10:18:20  2019-03-02 06:15:51
00038f9200  2018-12-07 11:03:50  2019-03-18 12:48:40
0003b55871  2018-11-23 16:26:57  2019-03-18 15:02:38
0004231e2a  2018-11-28 08:00:51  2019-03-17 12:19:22
0004254599  2018-11-24 14:29:33  2019-03-16 15:42:41
00042a927a  2018-12-02 17:01:25  2019-03-18 10:04:10
0004315e57  2018-11-25 13:20:56  2019-03-08 11:33:37
00047b3720  2018-11-30 11:04:35  2019-03-15 13:28:31
...                         ...                  ...
fff9f772f6  2018-12-16 17:20:58  2019-03-15 14:30:06
fffa2941b3  2018-11-22 12:23:49  2019-03-18 15:19:48
fffa7299cf  2018-11-25 08:29:57  2019-03-18 14:24:57
fffb19ee74  2018-12-06 08:03:23  2019-03-18 06:34:57
fffb23204d  2018-11-22 05:23:50  2019-03-18 05:43:16
fffb310977  2018-11-22 16:05:10  2019-03-18 15:06:06
fffb36a724  2018-12-26 18:29:15  2019-03-11 15:15:33
fffb52b456  2018-11-25 07:27:48  2019-03-09 13:37:43
fffb9a1eaa  2018-11-23 13:07:26  2019-03-16 16:21:53
fffbdaf8ed  2018-12-08 07:05:49  2019-03-13 12:50:44
fffbfc9346  2019-01-31 10:43:10  2019-03-18 11:16:00
fffc2e37ab  2019-02-28 13:29:56  2019-02-28 13:29:56
fffc2eadcb  2019-01-04 11:43:50  2019-03-10 15:45:04
fffc68800d  2018-12-02 09:16:40  2019-03-15 08:15:42
fffc6fd3ea  2019-02-08 17:29:01  2019-03-18 10:41:23
fffcb91f10  2018-12-03 12:57:49  2019-03-11 12:24:16
fffd5cd0c6  2018-11-29 14:58:57  2019-03-18 15:17:33
fffd63dfe3  2019-01-18 09:17:13  2019-03-13 08:54:48
fffd8c9d7d  2018-12-02 09:29:54  2019-03-08 13:06:17
fffde49bc6  2018-11-22 08:55:59  2019-03-04 12:38:47
fffe0abb97  2018-11-30 07:05:53  2019-03-15 06:35:50
fffe0ed719  2018-11-22 08:55:31  2019-03-18 14:23:11
fffea1204c  2018-12-02 12:25:55  2019-03-13 11:52:42
fffeacbe3e  2018-11-30 07:50:49  2019-03-15 09:29:59
fffeca6d22  2018-12-04 08:27:19  2019-03-15 14:46:39
fffece623e  2018-11-22 11:25:04  2019-03-13 15:44:39
ffff3dfff8  2018-11-25 16:14:36  2019-03-17 16:50:01
ffffaab9da  2019-01-18 14:22:44  2019-03-09 18:32:50
ffffeb5619  2019-02-13 14:55:11  2019-03-08 12:32:49
fffff6ce77  2018-11-29 17:34:42  2019-03-15 17:47:02

[400162 rows x 2 columns]